# Steps Done Already

### Downloading the Dataset and Uploading it to MongoDB

First, we downloaded the tsv file from UniProt website and created a python program to convert it to csv file. Since the size of the file was large, mongodb was not able to upload it to the database, therefore we split the database into 8 chunks of 10000 entries, and then fed them separately to mongodb



In [ ]:
import numpy as np
from itertools import combinations
import csv

In [ ]:
import pandas as pd

#df = pd.read_csv("data/uniprotkb_AND_model_organism_10090_2024_11_21.csv")
df = pd.read_csv("data/moodle_file.tsv", delimiter="\t")


Traceback (most recent call last):
  File "c:\Users\hackd\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'pandas'



In [ ]:
df.head()

Traceback (most recent call last):
  File "c:\Users\hackd\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'df' is not defined



In [ ]:
df.info()

Traceback (most recent call last):
  File "c:\Users\hackd\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'df' is not defined



In [ ]:
df.describe()

Traceback (most recent call last):
  File "c:\Users\hackd\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'df' is not defined



In [ ]:
df = df.drop_duplicates()
df.describe()

Traceback (most recent call last):
  File "c:\Users\hackd\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'df' is not defined



In [ ]:
print(df.isnull().sum())

Traceback (most recent call last):
  File "c:\Users\hackd\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'df' is not defined



In [ ]:
#df = df.dropna()
#df = df.dropna(axis=1)
#df.describe()

In [9]:
print(df.isnull().sum())

Entry                0
Entry Name           0
Protein names        0
Gene Names        3342
Organism             0
Sequence             0
EC number        75424
InterPro         12020
dtype: int64


In [ ]:
df['Gene Names'] = df['Gene Names'].fillna("Unknown")
df['EC number'] = df['EC number'].fillna("Not Available")
df = df.dropna(subset=['InterPro'])
print(df.isnull().sum())

Entry            0
Entry Name       0
Protein names    0
Gene Names       0
Organism         0
Sequence         0
EC number        0
InterPro         0
dtype: int64


In [10]:
df.describe()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro
count,73862,73862,73862,73862,73862,73862,73862,73862
unique,73862,73862,41087,33656,1,70969,1767,21481
top,A0A075F5C6,A0A075F5C6_MOUSE,Olfactory receptor,Unknown,Mus musculus (Mouse),WFLEYSTSECHFYNGTQRVRLLERYFYNLEENLRFDSDVGEFRAVT...,Not Available,IPR000276;IPR017452;IPR000725;
freq,1,1,1741,2444,73862,12,63447,1160


In [ ]:
df['Domains'] = df['InterPro'].apply(lambda x: ','.join(x.split(';')) if isinstance(x, str) else '')
df['InterPro'] = df['InterPro'].apply(lambda x: x.split(';') if isinstance(x, str) else [])
df.describe()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro,Domains
count,73862,73862,73862,73862,73862,73862,73862,73862,73862
unique,73862,73862,41087,33656,1,70969,1767,21481,21481
top,A0A075F5C6,A0A075F5C6_MOUSE,Olfactory receptor,Unknown,Mus musculus (Mouse),WFLEYSTSECHFYNGTQRVRLLERYFYNLEENLRFDSDVGEFRAVT...,Not Available,"[IPR000276, IPR017452, IPR000725, ]","IPR000276,IPR017452,IPR000725,"
freq,1,1,1741,2444,73862,12,63447,1160,1160


In [22]:
df.head()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro,Domains
0,A0A075F5C6,A0A075F5C6_MOUSE,Heat shock factor 1 (Heat shock transcription ...,Hsf1,Mus musculus (Mouse),MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...,Not Available,"[IPR000232, IPR027725, IPR010542, IPR036388, I...","IPR000232,IPR027725,IPR010542,IPR036388,IPR036..."
1,A0A087WPF7,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Auts2 Kiaa0442,Mus musculus (Mouse),MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,Not Available,"[IPR023246, ]","IPR023246,"
3,A0A087WRK1,A0A087WRK1_MOUSE,"Predicted gene, 20814 (Predicted gene, 20855) ...",Gm20905 Gm20814 Gm20835 Gm20850 Gm20855 Gm2086...,Mus musculus (Mouse),MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...,Not Available,"[IPR051443, IPR006888, ]","IPR051443,IPR006888,"
4,A0A087WRT4,A0A087WRT4_MOUSE,FAT atypical cadherin 1,Fat1,Mus musculus (Mouse),MGRHLTLLLLLLLFLQQFGDSDGSQRLEPTPPIQFTHFQYNVTVHE...,Not Available,"[IPR039808, IPR002126, IPR015919, IPR020894, I...","IPR039808,IPR002126,IPR015919,IPR020894,IPR013..."
5,A0A087WSP5,A0A087WSP5_MOUSE,Signal transducer and activator of transcription,Stat1,Mus musculus (Mouse),MSQWFELQQLDSKFLEQVHQLYDDSFPMEIRQYLAQWLEKQDWEHA...,Not Available,"[IPR008967, IPR000980, IPR036860, IPR001217, I...","IPR008967,IPR000980,IPR036860,IPR001217,IPR038..."


In [13]:
df

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro
0,A0A075F5C6,A0A075F5C6_MOUSE,Heat shock factor 1 (Heat shock transcription ...,Hsf1,Mus musculus (Mouse),MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...,Not Available,"[IPR000232, IPR027725, IPR010542, IPR036388, I..."
1,A0A087WPF7,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Auts2 Kiaa0442,Mus musculus (Mouse),MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,Not Available,"[IPR023246, ]"
3,A0A087WRK1,A0A087WRK1_MOUSE,"Predicted gene, 20814 (Predicted gene, 20855) ...",Gm20905 Gm20814 Gm20835 Gm20850 Gm20855 Gm2086...,Mus musculus (Mouse),MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...,Not Available,"[IPR051443, IPR006888, ]"
4,A0A087WRT4,A0A087WRT4_MOUSE,FAT atypical cadherin 1,Fat1,Mus musculus (Mouse),MGRHLTLLLLLLLFLQQFGDSDGSQRLEPTPPIQFTHFQYNVTVHE...,Not Available,"[IPR039808, IPR002126, IPR015919, IPR020894, I..."
5,A0A087WSP5,A0A087WSP5_MOUSE,Signal transducer and activator of transcription,Stat1,Mus musculus (Mouse),MSQWFELQQLDSKFLEQVHQLYDDSFPMEIRQYLAQWLEKQDWEHA...,Not Available,"[IPR008967, IPR000980, IPR036860, IPR001217, I..."
...,...,...,...,...,...,...,...,...
85874,Z4YN47,Z4YN47_MOUSE,SKI/DACH domain containing 1,Skida1,Mus musculus (Mouse),MGDLKSGFEEVDGVRLGYLIIKGKQMFALSQVFTDLLKNIPRTTVH...,Not Available,"[IPR009061, IPR052119, IPR003380, IPR037000, ]"
85875,Z4YN62,Z4YN62_MOUSE,Ataxin 7-like 3,Atxn7l3,Mus musculus (Mouse),EMSLSGLDNSKLEAIAQEIYADLVEDSCLGFCFEVHRAVKCGYFFL...,Not Available,"[IPR013246, IPR013243, IPR051078, ]"
85876,Z4YN74,Z4YN74_MOUSE,Transmembrane protein 11,Tmem11,Mus musculus (Mouse),MSVGMVSLSATDCYIVHEIYSGENAQDQFEYELEQALEAQYKYIVI...,Not Available,"[IPR026120, ]"
85878,Z4YN82,Z4YN82_MOUSE,Syntaphilin,Snph,Mus musculus (Mouse),MAMSLQGSRRASAGSRRRTSPPVSVRDAYGTSSLSSSSNSGSCKGS...,Not Available,"[IPR028197, ]"


In [ ]:
df = df.drop_duplicates(subset=["Entry"]).reset_index(drop=True)
df.describe()

KeyboardInterrupt: 

In [15]:
# df["Entry Name"] = df["Entry Name"].str.strip()
# df["Protein names"] = df["Protein names"].str.strip()
# df["Gene Names"] = df["Gene Names"].str.strip()

In [ ]:
# Separating the dataframe into two - one for nodes and the other for edges(relationships)
nodes_df = df[['Entry', 'Entry Name', 'Protein names', 'Gene Names', 'InterPro']]
#nodes_df.to_csv("nodes1.csv", index=False)
print("Completed")

Completed


In [ ]:
# Generating relationships using Jaccard Similarity
from itertools import combinations

edges = []

def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0

for (idx1, row1), (idx2, row2) in combinations(df.iterrows(), 2):
    set1 = set(row1['InterPro'])
    set2 = set(row2['InterPro'])
    similarity = jaccard_similarity(set1, set2)
    if similarity > 0:
            edges.append((row1['Entry'], row2['Entry'], similarity))
            
edges_df = pd.DataFrame(edges, columns = ["Protein1", "Protein2", "Weight"])
edges_df.to_csv("edges.csv", index=False)
print("Completed")

In [ ]:
import csv
import numpy as np
from itertools import combinations

df['InterPro'] = df['InterPro'].apply(set)

# Define chunk size
chunk_size = 1000

# Get all combinations of rows in chunks
combinations_list = list(combinations(df.iterrows(), 2))
total_combinations = len(combinations_list)
print(f"Total combinations: {total_combinations}")

# Split combinations into manageable chunks
chunked_combinations = np.array_split(combinations_list, total_combinations // chunk_size + 1)

def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0

with open("edges.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Protein1", "Protein2", "Weight"])

    for chunk in chunked_combinations:
        chunk_edges = []
        for (idx1, row1), (idx2, row2) in chunk:
            set1 = row1['InterPro']
            set2 = row2['InterPro']
            similarity = jaccard_similarity(set1, set2)
            if similarity > 0:
                chunk_edges.append((row1['Entry'], row2['Entry'], similarity))
        
        writer.writerows(chunk_edges)
        print(f"Processed and wrote chunk of size {len(chunk)}")


In [23]:
# Jaccard Similarity Function
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0


In [14]:
with open("edges2.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Protein1", "Protein2", "Weight"])  # Write header

    # Process combinations in manageable chunks
    batch_size = 500  # Adjust the batch size based on available memory
    combinations_generator = combinations(df.iterrows(), 2)  # Generate combinations

    for batch in range(0, len(df), batch_size):
        chunk_edges = []
        for (idx1, row1), (idx2, row2) in combinations(df[batch:batch + batch_size].iterrows(), 2):
            set1 = set(row1['InterPro'])
            set2 = set(row2['InterPro'])
            similarity = jaccard_similarity(set1, set2)
            if similarity > 0.8:
                chunk_edges.append((row1['Entry'], row2['Entry'], similarity))
        #break
        # Write the batch to the CSV file
        writer.writerows(chunk_edges)
        print(f"Processed batch {batch} - {batch + batch_size}")

Processed batch 0 - 500
Processed batch 500 - 1000
Processed batch 1000 - 1500
Processed batch 1500 - 2000
Processed batch 2000 - 2500
Processed batch 2500 - 3000
Processed batch 3000 - 3500
Processed batch 3500 - 4000
Processed batch 4000 - 4500
Processed batch 4500 - 5000
Processed batch 5000 - 5500
Processed batch 5500 - 6000
Processed batch 6000 - 6500
Processed batch 6500 - 7000
Processed batch 7000 - 7500
Processed batch 7500 - 8000
Processed batch 8000 - 8500
Processed batch 8500 - 9000
Processed batch 9000 - 9500
Processed batch 9500 - 10000
Processed batch 10000 - 10500
Processed batch 10500 - 11000
Processed batch 11000 - 11500
Processed batch 11500 - 12000
Processed batch 12000 - 12500
Processed batch 12500 - 13000
Processed batch 13000 - 13500
Processed batch 13500 - 14000
Processed batch 14000 - 14500
Processed batch 14500 - 15000
Processed batch 15000 - 15500
Processed batch 15500 - 16000
Processed batch 16000 - 16500
Processed batch 16500 - 17000
Processed batch 17000 - 

In [16]:
df2 = pd.read_csv("data/edges2.csv")
df2.describe()

,Weight
count,237724.000000
mean,0.974770
std,0.055031
min,0.809524
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [17]:
ec_df = df[['Entry', 'EC number']]
#ec_df.to_csv("data/ec_num.csv", index=False)
print("Completed")

Completed


### Statistics

In [19]:
ec_df.head(20)

,Entry,EC number
0,A0A075F5C6,Not Available
1,A0A087WPF7,Not Available
2,A0A087WRK1,Not Available
3,A0A087WRT4,Not Available
4,A0A087WSP5,Not Available
5,A0A088MLT8,Not Available
6,A0A0A6YXJ2,Not Available
7,A0A0A6YXR0,2.7.8.1; 2.7.8.2; 2.7.8.22
8,A0A0A6YXU9,Not Available
9,A0A0B4J1F4,Not Available


In [20]:
df.head(20)

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro,Domains
0,A0A075F5C6,A0A075F5C6_MOUSE,Heat shock factor 1 (Heat shock transcription ...,Hsf1,Mus musculus (Mouse),MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...,Not Available,"[IPR000232, IPR027725, IPR010542, IPR036388, I...","IPR000232,IPR027725,IPR010542,IPR036388,IPR036..."
1,A0A087WPF7,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Auts2 Kiaa0442,Mus musculus (Mouse),MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,Not Available,"[IPR023246, ]","IPR023246,"
2,A0A087WRK1,A0A087WRK1_MOUSE,"Predicted gene, 20814 (Predicted gene, 20855) ...",Gm20905 Gm20814 Gm20835 Gm20850 Gm20855 Gm2086...,Mus musculus (Mouse),MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...,Not Available,"[IPR051443, IPR006888, ]","IPR051443,IPR006888,"
3,A0A087WRT4,A0A087WRT4_MOUSE,FAT atypical cadherin 1,Fat1,Mus musculus (Mouse),MGRHLTLLLLLLLFLQQFGDSDGSQRLEPTPPIQFTHFQYNVTVHE...,Not Available,"[IPR039808, IPR002126, IPR015919, IPR020894, I...","IPR039808,IPR002126,IPR015919,IPR020894,IPR013..."
4,A0A087WSP5,A0A087WSP5_MOUSE,Signal transducer and activator of transcription,Stat1,Mus musculus (Mouse),MSQWFELQQLDSKFLEQVHQLYDDSFPMEIRQYLAQWLEKQDWEHA...,Not Available,"[IPR008967, IPR000980, IPR036860, IPR001217, I...","IPR008967,IPR000980,IPR036860,IPR001217,IPR038..."
5,A0A088MLT8,IQIP1_MOUSE,IQCJ-SCHIP1 readthrough transcript protein,Iqcj-Schip1 Iqschfp Schip1,Mus musculus (Mouse),MRLEELKRLQNPLEQVDDGKYLLENHQLAMDVENNIENYPLSLQPL...,Not Available,"[IPR029362, IPR039045, IPR015649, ]","IPR029362,IPR039045,IPR015649,"
6,A0A0A6YXJ2,A0A0A6YXJ2_MOUSE,IQ motif containing GTPase activating protein 3,Iqgap3,Mus musculus (Mouse),MEPCAAGRARTAYERLTAEEMDEQRRQNVAYQYLCRLEEAKRWMEV...,Not Available,"[IPR001715, IPR036872, ]","IPR001715,IPR036872,"
7,A0A0A6YXR0,A0A0A6YXR0_MOUSE,Choline/ethanolaminephosphotransferase 1 (EC 2...,Cept1,Mus musculus (Mouse),MSGHRSTRKRCGDSHPESPVGFGHMSTTGCVLNKLFQLPTPPLSRH...,2.7.8.1; 2.7.8.2; 2.7.8.22,"[IPR000462, IPR043130, IPR048254, IPR014472, ]","IPR000462,IPR043130,IPR048254,IPR014472,"
8,A0A0A6YXU9,A0A0A6YXU9_MOUSE,Transforming growth factor beta,Tgfb2,Mus musculus (Mouse),MHYCVLSTFLLLHLVPVALSLSTCSTLDMDQFMRKRIEAIRGQILS...,Not Available,"[IPR029034, IPR001839, IPR001111, IPR016319, I...","IPR029034,IPR001839,IPR001111,IPR016319,IPR015..."
9,A0A0B4J1F4,ARRD4_MOUSE,Arrestin domain-containing protein 4,Arrdc4,Mus musculus (Mouse),MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...,Not Available,"[IPR014752, IPR011021, IPR011022, IPR050357, I...","IPR014752,IPR011021,IPR011022,IPR050357,IPR014..."


In [22]:
df2.head(20)

,Protein1,Protein2,Weight
0,A0A075F5C6,D0VYS2,0.833333
1,A0A087WRT4,A0A1L1SQU7,1.000000
2,A0A087WRT4,F2Z4A3,1.000000
3,A0A0A6YXR0,D3YX12,1.000000
4,A0A0G2JDV3,A4UUI3,1.000000
5,A0A0R4J1M7,A2ASI5,0.888889
6,A0A0U1RNR0,E9Q4M2,0.833333
7,A0A142K3A7,E9Q9A6,1.000000
8,A0A1B0GT25,A5D6P3,0.888889
9,A0A1B0GT25,B2RX12,0.888889


#### ALL possible statistics

1. Proteins which have the most number of relationships - 293
2. Proteins which have the least number of relationships less than 2% - 4
3. Proteins which have no relationships(Jacardian similarity=0) - 0
4. Number of proteins which have similarities >= 80%
5. Number of proteins which have similarities < 80%
6. Similarities between isolated proteins
7. Find the number of unique proteins in the dataset with similarity >=0.8.Substract this number from the number of unique proteins in the initial dataset.
so these will be the isolated proteins(those who dont have similarity >=0.8)

In [ ]:
#Most Isolated Proteins

batch_size = 500  # Adjust the batch size based on available memory
combinations_generator = combinations(df.iterrows(), 2)  # Generate combinations

for batch in range(0, len(df), batch_size):
    chunk_edges = []
    for (idx1, row1), (idx2, row2) in combinations(df[batch:batch + batch_size].iterrows(), 2):
        set1 = set(row1['InterPro'])
        set2 = set(row2['InterPro'])
        similarity = jaccard_similarity(set1, set2)
        if similarity < 0.02:
            print(row1["Entry"], row2["Entry"])
            chunk_edges.append((row1['Entry'], row2['Entry'], similarity))

A0A0U1RNJ1 A2AAJ9
A0A0U1RNJ1 A2CG49
A0A0U1RNJ1 B2RQC6
A2CG49 B2RQC6


In [ ]:
# print(df2[df2['Protein1'] == 'A0A0U1RNJ1'])
# print(df2[df2['Protein2'] == 'A0A0U1RNJ1'])
# print(df2[df2['Protein1'] == 'B2RQC6'])
# print(df2[df2['Protein2'] == 'B2RQC6'])
# print(df2[df2['Protein1'] == 'A2CG49'])
# print(df2[df2['Protein2'] == 'A2CG49'])
# print(df2[df2['Protein1'] == 'A2AAJ9'])
# print(df2[df2['Protein2'] == 'A2AAJ9'])

Empty DataFrame
Columns: [Protein1, Protein2, Weight]
Index: []
Empty DataFrame
Columns: [Protein1, Protein2, Weight]
Index: []
Empty DataFrame
Columns: [Protein1, Protein2, Weight]
Index: []
Empty DataFrame
Columns: [Protein1, Protein2, Weight]
Index: []
Empty DataFrame
Columns: [Protein1, Protein2, Weight]
Index: []
Empty DataFrame
Columns: [Protein1, Protein2, Weight]
Index: []
Empty DataFrame
Columns: [Protein1, Protein2, Weight]
Index: []
Empty DataFrame
Columns: [Protein1, Protein2, Weight]
Index: []


In [39]:
protein1_max_count = df2['Protein1'].value_counts().head(10)
print(protein1_max_count)
protein2_max_count = df2['Protein2'].value_counts().head(10)
print(protein2_max_count)


Protein1
K7T918    295
K7T921    294
K7T926    293
K7T930    292
K7T935    291
K7T942    290
K7T952    288
K7T958    287
K7T977    286
K7T988    285
Name: count, dtype: int64
Protein2
K7U6K6    295
K7U6K0    294
K7U6J5    293
K7U6I8    292
K7U6I4    291
K7U6I0    290
K7U6H3    289
K7U6G6    288
K7U6G2    287
K7U6F6    286
Name: count, dtype: int64


In [42]:
df2[df2['Protein1']=='K7T977'].value_counts()

Protein1  Protein2  Weight  
K7T977    K7T988    1.000000    1
          K7TR97    1.000000    1
          K7TRE7    1.000000    1
          K7TRE2    1.000000    1
          K7TRD7    1.000000    1
                               ..
          K7TDT7    1.000000    1
          K7TDT9    1.000000    1
          K7TDU3    1.000000    1
          K7TDU6    0.857143    1
          K7U6K6    1.000000    1
Name: count, Length: 286, dtype: int64

In [43]:
df2[df2['Protein2']=='K7T977'].value_counts()

Protein1  Protein2  Weight
K7T918    K7T977    1.000     1
K7T921    K7T977    1.000     1
K7T926    K7T977    1.000     1
K7T930    K7T977    1.000     1
K7T935    K7T977    1.000     1
K7T942    K7T977    1.000     1
K7T947    K7T977    0.875     1
K7T952    K7T977    1.000     1
K7T958    K7T977    1.000     1
Name: count, dtype: int64

In [49]:
# Get value counts for each column
counts_1 = df2['Protein1'].value_counts().rename('Count_Protein1')
counts_2 = df2['Protein2'].value_counts().rename('Count_Protein2')

# Combine the counts into a single DataFrame
combined_counts = pd.concat([counts_1, counts_2], axis=1).fillna(0)

# Add a total count column
combined_counts['Total_Count'] = combined_counts['Count_Protein1'] + combined_counts['Count_Protein2']

for i in range(len(combined_counts)):
    # Check if the 'Total_Count' value in the current row equals 295.0
    if combined_counts.iloc[i]['Total_Count'] == 295.0:
        # Print the entire row
        print(combined_counts.iloc[i])


Count_Protein1    295.0
Count_Protein2      0.0
Total_Count       295.0
Name: K7T918, dtype: float64
Count_Protein1    294.0
Count_Protein2      1.0
Total_Count       295.0
Name: K7T921, dtype: float64
Count_Protein1    293.0
Count_Protein2      2.0
Total_Count       295.0
Name: K7T926, dtype: float64
Count_Protein1    292.0
Count_Protein2      3.0
Total_Count       295.0
Name: K7T930, dtype: float64
Count_Protein1    291.0
Count_Protein2      4.0
Total_Count       295.0
Name: K7T935, dtype: float64
Count_Protein1    290.0
Count_Protein2      5.0
Total_Count       295.0
Name: K7T942, dtype: float64
Count_Protein1    288.0
Count_Protein2      7.0
Total_Count       295.0
Name: K7T952, dtype: float64
Count_Protein1    287.0
Count_Protein2      8.0
Total_Count       295.0
Name: K7T958, dtype: float64
Count_Protein1    286.0
Count_Protein2      9.0
Total_Count       295.0
Name: K7T977, dtype: float64
Count_Protein1    285.0
Count_Protein2     10.0
Total_Count       295.0
Name: K7T988, dtype

In [53]:
# Filter rows where 'Total_Count' equals 295.0
filtered_rows = combined_counts[combined_counts['Total_Count'] == 295.0]

# pd.set_option('display.max_rows', None)  # Display all rows
# pd.set_option('display.max_columns', None)  # Display all columns
# pd.set_option('display.width', None)  # Remove column width limitation
# pd.set_option('display.colheader_justify', 'left')  # Align column headers to the left

# Print the filtered rows
print(filtered_rows)
print(len(filtered_rows))


        Count_Protein1  Count_Protein2  Total_Count
K7T918  295.0             0.0           295.0      
K7T921  294.0             1.0           295.0      
K7T926  293.0             2.0           295.0      
K7T930  292.0             3.0           295.0      
K7T935  291.0             4.0           295.0      
K7T942  290.0             5.0           295.0      
K7T952  288.0             7.0           295.0      
K7T958  287.0             8.0           295.0      
K7T977  286.0             9.0           295.0      
K7T988  285.0            10.0           295.0      
K7T993  284.0            11.0           295.0      
K7T999  283.0            12.0           295.0      
K7T9A5  282.0            13.0           295.0      
K7T9B2  281.0            14.0           295.0      
K7T9D0  280.0            15.0           295.0      
K7T9D7  279.0            16.0           295.0      
K7T9E3  278.0            17.0           295.0      
K7T9F1  277.0            18.0           295.0      
K7T9F6  276.

In [ ]:
print(df[df['Entry'] == 'A0A0U1RNJ1'])




   Entry       Entry Name         \
26  A0A0U1RNJ1  A0A0U1RNJ1_MOUSE   

   Protein names                                      Gene Names  \
26  Fatty acid synthase (EC 1.1.1.100) (EC 1.3.1.3...  Fasn        

   Organism              Sequence                                            \
26  Mus musculus (Mouse)  MEEVVIAGMSGKLPESENLQEFWANLIGGVDMVTDDDRRWKAGLYG...   

   EC number                                           \
26  1.1.1.100; 1.3.1.39; 2.3.1.38; 2.3.1.39; 2.3.1...   

   InterPro                                            \
26  [IPR029058, IPR001227, IPR036736, IPR014043, I...   

   Domains                                             
26  IPR029058,IPR001227,IPR036736,IPR014043,IPR016...  
26    1.1.1.100; 1.3.1.39; 2.3.1.38; 2.3.1.39; 2.3.1...
Name: EC number, dtype: object


In [ ]:
print(df[df['Entry'] == 'A2AAJ9'])



    Entry   Entry Name   Protein names                                       \
111  A2AAJ9  OBSCN_MOUSE  Obscurin (EC 2.7.11.1) (Obscurin-RhoGEF) (Obsc...   

    Gene Names   Organism               \
111  Obscn Gm878  Mus musculus (Mouse)   

    Sequence                                           EC number  \
111  MDHSFSGAPRFLTRPKAFVVSVGKDATLSCQIVGNPTPHVSWEKDR...  2.7.11.1   

    InterPro                                            \
111  [IPR035899, IPR000219, IPR003961, IPR036116, I...   

    Domains                                             
111  IPR035899,IPR000219,IPR003961,IPR036116,IPR007...  


In [59]:
print(df[df['Entry'] == 'A2CG49'])


    Entry   Entry Name   Protein names                                       \
191  A2CG49  KALRN_MOUSE  Kalirin (EC 2.7.11.1) (Protein Duo) (Serine/th...   

    Gene Names Organism               \
191  Kalrn      Mus musculus (Mouse)   

    Sequence                                           EC number  \
191  MVLSGSFRNDGLKASDVLPILKEKVAFVSGGRDKRGGPILTFPARS...  2.7.11.1   

    InterPro                                            \
191  [IPR001251, IPR036865, IPR035899, IPR000219, I...   

    Domains                                             
191  IPR001251,IPR036865,IPR035899,IPR000219,IPR003...  


In [60]:
print(df[df['Entry'] == 'B2RQC6'])

    Entry   Entry Name  Protein names                                       \
269  B2RQC6  PYR1_MOUSE  Multifunctional protein CAD (Carbamoyl phospha...   

    Gene Names Organism               \
269  Cad        Mus musculus (Mouse)   

    Sequence                                            \
269  MAALVLEDGSVLQGRPFGAAVSTAGEVVFQTGMVGYPEALTDPSYK...   

    EC number                                      \
269  2.1.3.2; 3.5.1.2; 3.5.2.3; 6.3.4.16; 6.3.5.5   

    InterPro                                            \
269  [IPR006680, IPR006132, IPR006130, IPR036901, I...   

    Domains                                             
269  IPR006680,IPR006132,IPR006130,IPR036901,IPR002...  


In [ ]:
filtered_rows2 = combined_counts[combined_counts['Total_Count'] == 50.0]
print(filtered_rows2)

Traceback (most recent call last):
  File "c:\Users\hackd\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'combined_counts' is not defined

